In [1]:
from korean_crawling_bundles import *
from konlpy.tag import *
import datetime
from IPython.display import display

import re
import numpy as np
import pandas as pd
from pprint import pprint
from konlpy.tag import Hannanum

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stopwords는 불필요한 단어, 즉 조사나 관사들을 없애는 툴이다.

In [2]:
stop_words = list(pd.read_csv('kor_stop_words.csv').T.iloc[:1,:].values[0])
stop_words[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

# 전체 데이터프레임 병합하기

In [3]:
naver_df = pd.read_csv('{}_with_link_naver.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False))
daum_df = pd.read_csv('{}_with_link_daum.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False))
daum_df = daum_df.loc[:,['title','link','content']]

df = pd.concat([naver_df,daum_df],axis=0)
df.reset_index(drop=True,inplace=True)
df.tail()

,title,link,content
784,"[앵커의 눈] ""수수료 때문에 못살겠다"" vs ""인하 여력 없다""..정부는 '진퇴양난'",http://m.finance.daum.net/m/news/news_content....,[앵커] 가맹점들이 내는 카드수수료는 연매출액에 따라 이렇게 세 구간으로 나뉘어있습...
785,영국 3분기 임금 3.2% 증가..금융위기 후 최고,http://m.finance.daum.net/m/news/news_content....,(런던=연합뉴스) 박대한 특파원 = 영국의 3분기 임금 상승률이 2008년 금융위기...
786,"OPEC, 2019년 석유 공급과잉 경고..감산 바닥다지기?",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 이창규 기자 = 석유수출국기구(OPEC)는 13일 세계 경제가 둔화...
787,또 감원 바람 부나.. 조선업계 뒤숭숭,http://m.finance.daum.net/m/news/news_content....,연말을 앞두고 조선업계에 또다시 ‘구조조정’ 바람이 불 조짐이다. 국내 조선사 ‘빅...
788,"李총리 ""캐나다 세이프가드 조치 우려..양국간 공정무역 지속""",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 박승주 기자 = 이낙연 국무총리는 13일 오후 정부서울청사에서 조지...


In [4]:
append_ls,append_idx = [],[]
for idx,i in enumerate(df['title']) : 
    if i not in append_ls:
        append_ls.append(i)
        append_idx.append(idx)

In [5]:
df = df.loc[append_idx]
# df.reset_index(drop=True,inplace=True)
df.tail()

,title,link,content
739,"매매 끊기고·경매는 늘고..쌓이는 대세 하락 조짐, 서울 집값 향방은?",http://m.finance.daum.net/m/news/news_content....,[아시아경제 주상돈 기자] 서울 부동산 시장에 관망세가 짙어지고 있다. 아파트 거래...
740,환경오염 없이 수소연료 만드는 고성능 촉매 개발,http://m.finance.daum.net/m/news/news_content....,(대전=연합뉴스) 이재림 기자 = 국내 연구진이 환경오염 없이 수소연료를 생산할 수...
741,强달러 장기화되나? 유럽발 정치 불확실성..1140원에 바짝 다가선 원·달러 환율,http://m.finance.daum.net/m/news/news_content....,NaN
742,납품가 후려치기 등 불공정 기업 벌점 최대 2배로,http://m.finance.daum.net/m/news/news_content....,[한겨레] 하청기업에게 납품단가를 부당하게 깎거나 납품대금을 제때 주지 않는 등 ...
743,"자외선차단제 등 27개 제품, '미세먼지 차단 효과' 없어",http://m.finance.daum.net/m/news/news_content....,식품의약품안전처는 화장품 중 미세먼지 차단·세정에 효과가 있다고 광고·판매하는 자외...


In [6]:
df.fillna('0',inplace=True)

In [7]:
df['content'] = [i.replace('  ','') for i in df['content'].values]

________

In [8]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# # ▶ 이후로 나온 단어들은 기사의 author 에 관한 내용으로 지워준다.
# data = [data[i][:re.search('▶',data[i]).start()] if '▶' in i else data[i] for i in range(len(data))]

pprint(data[:1])

['하나카드는 지난 13일 전국사무금융서비스노동조합(이하 사무금융노조)과 불평등 양극화 해소를 위한 사회연대기금 출연 조인식을 개최했다고 '
 '14일 밝혔다.하나카드 노사는 오는 2020년까지 올해 2억원 등 3년간 총 6억원의 사회연대기금을 출연할 예정이다. 이날 조인식에는 '
 '정수진 하나카드 대표이사 김현정 사무금융노조 위원장 정종우 사무금융노조 하나외환카드지부장이 참석했다.이번 하나카드 노사의 사회연대기금 '
 '출연은 지난 4월 사무금융노조가 기금 조성 선포식을 개최한 이후 진행된 다섯번째 합의다. 사무금융노조는 지난 2월 개최된 대의원 대회를 '
 '통해 노동조합의 사회적 책임과 역할을 강조하며 불평등 양극화 해소를 위한 특별위원회를 발족했다.정수진 하나카드 대표이사는 “이번 조인식을 '
 '통해 사회적 약자 및 나라의 미래인 청년들에게 많은 도움이 되길 기대한다”며 “그 동안 하나카드는 하나금융그룹과 함께 하나데이행사 '
 '독거노인 돕기 등 기업의 사회적 소명을 위한 다양한 사회공헌 활동을 해왔는데 이번에는 노사와 함께 하게 돼 더욱 뜻 깊다”고 말했다.현재 '
 '사무금융노사는 업종본부별 지부별 실무교섭을 진행 중이며 지난 KB증권 KB카드 애큐온저축은행 교보증권 하나카드에 이어 사회연대기금 출연 '
 '합의가 진행될 전망이다. 이와 함께 현재는 노사가 함께 실무 추진단을 함께 구성해 활동하고 있으며 공익재단 사무금융연대기금(가칭) 출범을 '
 '준비하고 있다.※ 저작권자 ⓒ 파이낸셜뉴스. 무단 전재-재배포 금지']


# tokenization process

- 정규식 표현을 통해서 문장 내에 이메일과 기타 특수 문자들을 없애주었지만, 여전히 난잡해보인다.
- LDA 알고리즘을 사용하기 위해서는, 문장들을 단어들의 묶음으로 변환시켜주는 과정이 필요하다.
- 이러한 과정을 Tokenization 이라고 한다.

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations
        # 구두점(말끝에 찍는 쉼표나 점들을 의미) 을 없애주는 것이다.

data_words = list(sent_to_words(data))

print(data_words[:1])

[['하나카드는', '지난', '전국사무금융서비스노동조합', '이하', '사무금융노조', '불평등', '양극화', '해소를', '위한', '사회연대기금', '출연', '조인식을', '개최했다고', '밝혔다', '하나카드', '노사는', '오는', '년까지', '올해', '억원', '년간', '억원의', '사회연대기금을', '출연할', '예정이다', '이날', '조인식에는', '정수진', '하나카드', '대표이사', '김현정', '사무금융노조', '위원장', '정종우', '사무금융노조', '하나외환카드지부장이', '참석했다', '이번', '하나카드', '노사의', '사회연대기금', '출연은', '지난', '사무금융노조가', '기금', '조성', '선포식을', '개최한', '이후', '진행된', '다섯번째', '합의다', '사무금융노조는', '지난', '개최된', '대의원', '대회를', '통해', '노동조합의', '사회적', '책임과', '역할을', '강조하며', '불평등', '양극화', '해소를', '위한', '특별위원회를', '발족했다', '정수진', '하나카드', '대표이사는', '이번', '조인식을', '통해', '사회적', '약자', '나라의', '미래인', '청년들에게', '많은', '도움이', '되길', '기대한다', '동안', '하나카드는', '하나금융그룹과', '함께', '하나데이행사', '독거노인', '돕기', '기업의', '사회적', '소명을', '위한', '다양한', '사회공헌', '활동을', '해왔는데', '이번에는', '노사와', '함께', '하게', '더욱', '깊다', '말했다', '현재', '사무금융노사는', '업종본부별', '지부별', '실무교섭을', '진행', '중이며', '지난', 'kb증권', 'kb카드', '애큐온저축은행', '교보증권', '하나카드에', '이어', '사회연대기금', '출연', '합의가', '진행될', '전망이다', '이와', '함께', '현재는', '노사가', '함께', '실무', '추진단을', '함께',

### Bigram , Trigram 모델 만들기
- Bigram : 문서에서 함께 자주 등장하는 2개의 단어
- Trigram : 문서에서 함께 자주 등장하는 3개의 단어
- ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.
- Phrases : 모델을 빌드한다.
- min_count , threshold : Pharases 의 중요한 두 개의 파라미터
    - min_count (float, optional) : Ignore all words and bigrams with total collected count lower than this value.
    - threshold (float, optional) : Represent a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. Heavily depends on concrete scoring-function, see the scoring parameter.

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/home/hskimim/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['하나카드는', '지난', '전국사무금융서비스노동조합', '이하', '사무금융노조', '불평등_양극화_해소를_위한', '사회연대기금_출연', '조인식을', '개최했다고_밝혔다', '하나카드', '노사는', '오는', '년까지', '올해', '억원', '년간', '억원의', '사회연대기금을', '출연할', '예정이다', '이날', '조인식에는', '정수진_하나카드', '대표이사', '김현정_사무금융노조', '위원장', '정종우', '사무금융노조', '하나외환카드지부장이', '참석했다', '이번', '하나카드', '노사의', '사회연대기금', '출연은', '지난', '사무금융노조가', '기금', '조성', '선포식을', '개최한', '이후', '진행된', '다섯번째', '합의다', '사무금융노조는', '지난', '개최된', '대의원', '대회를', '통해', '노동조합의', '사회적', '책임과', '역할을', '강조하며', '불평등_양극화_해소를_위한', '특별위원회를', '발족했다', '정수진_하나카드', '대표이사는', '이번', '조인식을', '통해', '사회적', '약자', '나라의', '미래인', '청년들에게', '많은_도움이', '되길_기대한다', '동안', '하나카드는', '하나금융그룹과', '함께', '하나데이행사', '독거노인', '돕기', '기업의', '사회적', '소명을', '위한', '다양한', '사회공헌_활동을', '해왔는데', '이번에는', '노사와', '함께', '하게', '더욱', '깊다', '말했다', '현재', '사무금융노사는', '업종본부별', '지부별', '실무교섭을', '진행', '중이며', '지난', 'kb증권', 'kb카드', '애큐온저축은행', '교보증권', '하나카드에', '이어', '사회연대기금_출연', '합의가', '진행될', '전망이다', '이와_함께', '현재는', '노사가', '함께', '실무', '추진단을', '함께', '구성해', '활동하고', '있으며', '공익재단', '사무금융연대기금', '가칭', 

In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized = data_words_bigrams.copy()

print(data_lemmatized[:1])

[['하나카드는', '지난', '전국사무금융서비스노동조합', '이하', '사무금융노조', '불평등_양극화', '해소를_위한', '사회연대기금_출연', '조인식을', '개최했다고_밝혔다', '하나카드', '노사는', '오는', '년까지', '올해', '억원', '년간', '억원의', '사회연대기금을', '출연할', '예정이다', '이날', '조인식에는', '정수진_하나카드', '대표이사', '김현정_사무금융노조', '위원장', '정종우', '사무금융노조', '하나외환카드지부장이', '참석했다', '하나카드', '노사의', '사회연대기금', '출연은', '지난', '사무금융노조가', '기금', '조성', '선포식을', '개최한', '이후', '진행된', '다섯번째', '합의다', '사무금융노조는', '지난', '개최된', '대의원', '대회를', '통해', '노동조합의', '사회적', '책임과', '역할을', '강조하며', '불평등_양극화', '해소를_위한', '특별위원회를', '발족했다', '정수진_하나카드', '대표이사는', '조인식을', '통해', '사회적', '약자', '나라의', '미래인', '청년들에게', '많은_도움이', '되길_기대한다', '하나카드는', '하나금융그룹과', '하나데이행사', '독거노인', '돕기', '기업의', '사회적', '소명을', '위한', '다양한', '사회공헌_활동을', '해왔는데', '이번에는', '노사와', '하게', '더욱', '깊다', '말했다', '현재', '사무금융노사는', '업종본부별', '지부별', '실무교섭을', '진행', '중이며', '지난', 'kb증권', 'kb카드', '애큐온저축은행', '교보증권', '하나카드에', '이어', '사회연대기금_출연', '합의가', '진행될', '전망이다', '이와', '현재는', '노사가', '실무', '추진단을', '구성해', '활동하고', '있으며', '공익재단', '사무금융연대기금', '가칭', '출범을', '준비하고', '저작권자_파이낸셜뉴스', '무단_전재',

- LDA 모델에 들어가야 하는 두 가지 입력변수는 딕셔너리와(id2word) 코퍼스(corpus)이다.
- gensim 은 문서 내에 있는 단어별로 유니크한 아이디를 할당해준다.
- 아래의 각각의 엘리먼트 튜플당 의미하는 것은 [word_id,word_frequency] 이다.

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 3), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 4), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 2), (100, 2), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1)]]


만약 해당 id에 속한 단어를 보고싶으면

In [14]:
id2word[0]

'kb증권'

위의 표는 컴퓨터가 읽기 쉽게끔 만들어준 것이고, Counter 객체처럼 사람이 읽기 쉽게 만든 것은 아래와 같다.

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('kb증권', 1),
  ('kb카드', 1),
  ('가칭', 1),
  ('강조하며', 1),
  ('개최된', 1),
  ('개최한', 1),
  ('개최했다고_밝혔다', 1),
  ('공익재단', 1),
  ('교보증권', 1),
  ('구성해', 1),
  ('기금', 1),
  ('기업의', 1),
  ('김현정_사무금융노조', 1),
  ('깊다', 1),
  ('나라의', 1),
  ('년간', 1),
  ('년까지', 1),
  ('노동조합의', 1),
  ('노사가', 1),
  ('노사는', 1),
  ('노사와', 1),
  ('노사의', 1),
  ('다섯번째', 1),
  ('다양한', 1),
  ('대의원', 1),
  ('대표이사', 1),
  ('대표이사는', 1),
  ('대회를', 1),
  ('더욱', 1),
  ('독거노인', 1),
  ('돕기', 1),
  ('되길_기대한다', 1),
  ('많은_도움이', 1),
  ('말했다', 1),
  ('무단_전재', 1),
  ('미래인', 1),
  ('발족했다', 1),
  ('불평등_양극화', 2),
  ('사무금융노사는', 1),
  ('사무금융노조', 2),
  ('사무금융노조가', 1),
  ('사무금융노조는', 1),
  ('사무금융연대기금', 1),
  ('사회공헌_활동을', 1),
  ('사회연대기금', 1),
  ('사회연대기금_출연', 2),
  ('사회연대기금을', 1),
  ('사회적', 3),
  ('선포식을', 1),
  ('소명을', 1),
  ('실무', 1),
  ('실무교섭을', 1),
  ('애큐온저축은행', 1),
  ('약자', 1),
  ('억원', 1),
  ('억원의', 1),
  ('업종본부별', 1),
  ('역할을', 1),
  ('예정이다', 1),
  ('오는', 1),
  ('올해', 1),
  ('위원장', 1),
  ('위한', 1),
  ('이날', 1),
  ('이번에는', 1),
  ('이어', 1),
  

여태까지 해온 것이 LDA 모델 생성에 필요한 것들을 전부 한 것이다. 코퍼스와 딕셔너리를 생성한 것에 더해서, 우리는 몇 개의 토픽을 할당할 것인지에 대한 결정을 해주어야 한다.

- alpha , eta 는 토픽들의 떨어진 정도(sparsity)에 영향을 끼치는 하이퍼 파라미터이다. 도큐먼트에 따르면, 디폴트값은 1.0/num_topics prior 이다.
- chunksize 는 각각의 training chunk 에 사용될 문서의 갯수를 의미한다. 확실하지는 않지만, batch_size 와 유사한 의미를 갖는 것로 해석된다.

    - IN ADDITION : Text chunking, also referred to as shallow parsing, is a task that follows Part-Of-Speech Tagging and that adds more structure to the sentence. The result is a grouping of the words in “chunks”.


In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

위의 모델을 통해 반환되는 것은 토픽의 수는 20개이고 각각의 키워드(단어들의 집합)와 키워드들 간의 조합이 특정한 토픽의 가중치를 정해주는데 기여하는 것이다.

- lda_model 객체에 print_topics 메소드를 operating 하면, 각각의 키워드들이 토픽에 기여하는 가중치(importance)를 알 수 있다.
- 0부터 19까지 총 20개에 해당하는 토픽이 있는 것을 알 수 있고, 각각의 토픽에 위치해있는 키워드들과 이들 키워드들이 해당 토픽에서 가지는 중요도가 순서대로 나와있다.

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"미세먼지" + 0.008*"차단" + 0.005*"없는" + 0.005*"제품" + 0.005*"효과가" + '
  '0.005*"자료" + 0.005*"판매하는" + 0.005*"실증자료" + 0.005*"광고" + 0.004*"등을"'),
 (1,
  '0.007*"대한" + 0.006*"점에서" + 0.006*"점으로" + 0.005*"밝혔다" + 0.005*"통해" + '
  '0.005*"불공정행위" + 0.005*"벌점" + 0.004*"기존" + 0.004*"중소기업" + 0.003*"기업에"'),
 (2,
  '0.005*"통해" + 0.005*"위해" + 0.004*"있는" + 0.004*"촉매" + 0.004*"위한" + 0.004*"백금" '
  '+ 0.004*"나노입자" + 0.004*"kbsi" + 0.004*"말했다" + 0.004*"기자"'),
 (3,
  '0.004*"대한" + 0.003*"지난" + 0.003*"있는" + 0.002*"기자" + 0.002*"정부의" + '
  '0.002*"경우" + 0.002*"삼성바이오" + 0.002*"것은" + 0.002*"현재" + 0.002*"한다"'),
 (4,
  '0.007*"억원" + 0.006*"이후" + 0.005*"서울" + 0.005*"있는" + 0.004*"기자" + 0.004*"지난" '
  '+ 0.004*"아파트" + 0.004*"것으로" + 0.004*"평균" + 0.004*"분기"')]


In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.895509745951102

Coherence Score:  0.5636501895287542


pyLDAvis 만큼 jupyter notebook에서 LDA랄 잘 작동하면서 시각화하는 툴도 없다.

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      28.698589        1       1 -0.234266 -0.002718
3      27.872933        1       2 -0.014348 -0.062995
2      15.708000        1       3  0.041848  0.150487
1      14.740029        1       4  0.099731 -0.169156
0      12.980450        1       5  0.107035  0.084383, topic_info=      Category        Freq       Term       Total  loglift  logprob
term                                                               
20779  Default  191.000000       미세먼지  191.000000  30.0000  30.0000
34103  Default  122.000000         차단  122.000000  29.0000  29.0000
8471   Default  104.000000        점으로  104.000000  28.0000  28.0000
2968   Default  112.000000        점에서  112.000000  27.0000  27.0000
54     Default  321.000000         억원  321.000000  26.0000  26.0000
9428   Default  136.000000        아파트  136.000000  25.0000  25.0000
2299   Default  132.000000         분기  132.000000  24.0000  24.0000
3270   Default  129.000000        줄었다  129.000000  23.0000  23.0000
1221   Default   83.000000        효과가   83.000000  22.0000  22.0000
23773  Default   81.000000         자료   81.000000  21.0000  21.0000
1885   Default   76.000000       판매하는   76.000000  20.0000  20.0000
1530   Default  151.000000         없는  151.000000  19.0000  19.0000
12543  Default   88.000000         제품   88.000000  18.0000  18.0000
34089  Default   73.000000       실증자료   73.000000  17.0000  17.0000
34076  Default   73.000000         광고   73.000000  16.0000  16.0000
34048  Default   78.000000      불공정행위   78.000000  15.0000  15.0000
34045  Default   78.000000         벌점   78.000000  14.0000  14.0000
4617   Default   77.000000         촉매   77.000000  13.0000  13.0000
33975  Default   76.000000         백금   76.000000  12.0000  12.0000
33955  Default   76.000000       kbsi   76.000000  11.0000  11.0000
33965  Default   76.000000       나노입자   76.000000  10.0000  10.0000
399    Default  144.000000         평균  144.000000   9.0000   9.0000
93     Default  249.000000         통해  249.000000   8.0000   8.0000
68     Default  223.000000         이후  223.000000   7.0000   7.0000
473    Default  434.000000         대한  434.000000   6.0000   6.0000
3446   Default  102.000000         또는  102.000000   5.0000   5.0000
429    Default  196.000000         서울  196.000000   4.0000   4.0000
14992  Default   56.000000         유통   56.000000   3.0000   3.0000
3999   Default   65.000000       중소기업   65.000000   2.0000   2.0000
1371   Default   94.000000       증가했다   94.000000   1.0000   1.0000
...        ...         ...        ...         ...      ...      ...
34091   Topic5   24.489899      실증자료로   25.087697   2.0176  -6.4086
34101   Topic5   24.489899       제출받아   25.087697   2.0176  -6.4086
34074   Topic5   24.489899        개소에   25.087697   2.0176  -6.4086
14992   Topic5   54.568597         유통   56.494461   2.0070  -5.6074
809     Topic5   24.656358        효과에   25.261549   2.0175  -6.4018
34088   Topic5   24.489899  식품의약품안전처는   25.087697   2.0176  -6.4086
23773   Topic5   76.532034         자료   81.192242   1.9826  -5.2691
34080   Topic5   24.489899        보습제   25.087697   2.0176  -6.4086
34079   Topic5   24.489899        민승기   25.087697   2.0176  -6.4086
34093   Topic5   24.489899         원료   25.087697   2.0176  -6.4086
1221    Topic5   76.713044        효과가   83.731254   1.9542  -5.2668
12543   Topic5   78.911787         제품   88.345810   1.9288  -5.2385
34087   Topic5   24.489899       식약처는   25.087697   2.0176  -6.4086
1105    Topic5   53.437652         결과   64.254247   1.8574  -5.6283
34078   Topic5   24.489899       광고하는   25.087697   2.0176  -6.4086
1494    Topic5   50.241630        내용이   68.200930   1.7361  -5.6900
1530    Topic5   79.011382         없는  151.429235   1.3912  -5.2372
184     Topic5   51.302554         아닌   79.013074   1.6099  -5.6691
3364    Topic5   26.638708        인터넷   28.472152   1.9752  -6.3245
3446    Topic5   56.915233         또는  102.

- Gensim 의 LDA 알고리즘보다 Mallet 버젼이 더 나은 퀄리티를 보여준다.
- https://www.machinelearningplus.com/wp-content/uploads/2018/03/mallet-2.0.8.zip 깔고 해당 경로를 아래에 넣어주면 된다.

In [20]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/home/hskimim/Documents/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)
# Show Topics
pprint(ldamallet.show_topics(formatted=False))


[(0,
  [('위한', 0.008159523263809306),
   ('통해', 0.008022003208801283),
   ('위해', 0.007884483153793262),
   ('한다', 0.005592482236992895),
   ('있는', 0.00531744212697685),
   ('말했다', 0.005088242035296814),
   ('등을', 0.004904881961952785),
   ('기자', 0.004767361906944763),
   ('위원장은', 0.004675681870272748),
   ('이날', 0.004629841851936741)]),
 (1,
  [('대한', 0.009084979500559075),
   ('지난', 0.008339545285128588),
   ('이날', 0.004379426015654118),
   ('관련', 0.003866939992545658),
   ('서울_뉴스', 0.0036339918002236305),
   ('기자', 0.003587402161759225),
   ('경우', 0.003587402161759225),
   ('당시', 0.003354453969437197),
   ('삼성바이오', 0.002981736861721953),
   ('대해', 0.002981736861721953)]),
 (2,
  [('있는', 0.0058713741348471885),
   ('내년', 0.004665992689944786),
   ('이라고', 0.00439380978303134),
   ('미국', 0.004043860331285481),
   ('것으로', 0.003849443969204448),
   ('하는', 0.003771677424372035),
   ('현재', 0.003616144334707209),
   ('것은', 0.003538377789874796),
   ('것이다', 0.003538377789874796),
   ('있다는', 0

In [21]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.376275411171716


### 항상 더 우월한 것은 아닌듯

#### 해당 문장에서 지배적인 토픽을 찾기
- 토픽 모델링의 주요 활용점은 해당 문서의 토픽이 무엇이냐에 관한 것이다.
- 이를 알아내기 위해서는 해당 문서에서 가장 기여를 많이 한, 즉 중요도가 가장 높은 토픽의 넘버를 찾아야 한다.
- format_topics_sentences() 메소드는 보여지는 테이블로 훌륭하게 정보를 병합해준다.

In [22]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet , corpus=corpus, texts=data)

#ldamallet 파라미터를 optimal_params 로 바꿔주는 시퀀스로 업데이트할 것!

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.5536,"위한, 통해, 위해, 한다, 있는, 말했다, 등을, 기자, 위원장은, 이날",하나카드는 지난 13일 전국사무금융서비스노동조합(이하 사무금융노조)과 불평등 양극화...
1,1,3.0,0.7063,"통해, 밝혔다, 재배포_금지, 관계자는, 있는, 기자, 억원을, 직접, 금지, 무단_전재",현대카드는 오는 23일 시작되는 블랙프라이데이를 맞아 해외직구 이용 시 안전한 카드...
2,2,3.0,0.6378,"통해, 밝혔다, 재배포_금지, 관계자는, 있는, 기자, 억원을, 직접, 금지, 무단_전재",KB국민카드는 14일 소아암 어린이 돕기 임직원 헌혈 행사를 가졌다고 밝혔다.KB국...
3,3,1.0,0.5470,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",롯데카드는 그랜저IG 100대를 경품으로 증정하는 ‘롯데 엘드림 페스타’(L.Dre...
4,4,0.0,0.4363,"위한, 통해, 위해, 한다, 있는, 말했다, 등을, 기자, 위원장은, 이날",IBK저축은행은 지역 내 우수 학생에게 총 700만원의 장학금 수여했다고 14일 밝...
5,5,3.0,0.6031,"통해, 밝혔다, 재배포_금지, 관계자는, 있는, 기자, 억원을, 직접, 금지, 무단_전재",KB국민카드는 세계 3대 신용평가사 중 하나인 ‘피치’(Fitch)로부터 2년 연속...
6,6,3.0,0.5222,"통해, 밝혔다, 재배포_금지, 관계자는, 있는, 기자, 억원을, 직접, 금지, 무단_전재",금융감독원이 가맹점 카드수수료 개편안 발표를 앞둔 시점에서 전업계 8개 카드사 검사...
7,7,3.0,0.5743,"통해, 밝혔다, 재배포_금지, 관계자는, 있는, 기자, 억원을, 직접, 금지, 무단_전재",<아이뉴스24>[아이뉴스24 유재형 기자] 전국은행연합회 소속 18개 금융기관이 3...
8,8,1.0,0.3431,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",(인천=뉴스1) 정진욱 기자 = 14일 오후 인천시 송도의 삼성바이오로직스 공장앞 ...
9,9,1.0,0.3627,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",(인천=뉴스1) 정진욱 기자 = 14일 오후 인천시 송도의 삼성바이오로직스 공장앞 ...


In [23]:
recommendation_df = df_dominant_topic.sort_values(by='Topic_Perc_Contrib',ascending=False)
recommendation_df.reset_index(drop=True,inplace=True)
recommendation_df

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,200,1.0,0.8763,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해","회계처리 변경 고의성 인정 여부 주목…삼바 ""본질을 봐야 한다"" (서울=연합뉴스) ..."
1,617,1.0,0.8715,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",(서울=연합뉴스) 박상돈 기자 = 금융위원회 산하 증권선물위원회(이하 증선위)가 1...
2,246,1.0,0.8674,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",회계처리 변경 고의성 인정 여부 주목 (서울=연합뉴스) 박상돈 기자 = 금융위원회 ...
3,634,1.0,0.8596,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",(서울=연합뉴스) 박상돈 기자 = 금융위원회 산하 증권선물위원회(이하 증선위)가 1...
4,632,1.0,0.8310,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",(서울=뉴스1) 김태헌 기자 = 삼성바이오로직스(삼성바이오) 분식회계 혐의에 대한 ...
5,238,1.0,0.8283,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",(서울=뉴스1) 김태헌 기자 = 삼성바이오로직스(삼성바이오) 분식회계 혐의에 대한 ...
6,614,4.0,0.8245,"억원, 올해, 분기, 이후, 지난, 지난해, 지난달, 대비, 기자, 것으로",(세종=연합뉴스) 민경락 이대희 기자 = 지난달에도 고용시장의 어려움이개선되지 못하...
7,305,4.0,0.8148,"억원, 올해, 분기, 이후, 지난, 지난해, 지난달, 대비, 기자, 것으로",[ 오정민 기자] 지난달 금융권 가계대출이 10조4000억원 증가했다. 특히 은행권...
8,254,1.0,0.8141,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해",【서울=뉴시스】김형섭 기자 = 금융위원회 산하 증권선물위원회는 14일 삼성바이오로직...
9,442,0.0,0.8126,"위한, 통해, 위해, 한다, 있는, 말했다, 등을, 기자, 위원장은, 이날",[이데일리 박일경 기자] 신한금융그룹과 SK가 사회적 가치 창출을 위한 생태계를 조...


In [24]:
duplicate_ls = []
duplicate_idx = []
for idx,val in enumerate(recommendation_df['Dominant_Topic'].values):
    if val not in duplicate_ls:
        duplicate_ls.append(val)
        duplicate_idx.append(idx)
        if len(duplicate_ls) == 5 : break

In [25]:
print(duplicate_ls)
print(duplicate_idx)

[1.0, 4.0, 0.0, 2.0, 3.0]
[0, 6, 9, 28, 60]


In [26]:
display(recommendation_df.loc[duplicate_idx])
display(df.loc[recommendation_df.loc[duplicate_idx]['Document_No'].values])

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,200,1.0,0.8763,"대한, 지난, 이날, 관련, 서울_뉴스, 기자, 경우, 당시, 삼성바이오, 대해","회계처리 변경 고의성 인정 여부 주목…삼바 ""본질을 봐야 한다"" (서울=연합뉴스) ..."
6,614,4.0,0.8245,"억원, 올해, 분기, 이후, 지난, 지난해, 지난달, 대비, 기자, 것으로",(세종=연합뉴스) 민경락 이대희 기자 = 지난달에도 고용시장의 어려움이개선되지 못하...
9,442,0.0,0.8126,"위한, 통해, 위해, 한다, 있는, 말했다, 등을, 기자, 위원장은, 이날",[이데일리 박일경 기자] 신한금융그룹과 SK가 사회적 가치 창출을 위한 생태계를 조...
28,429,2.0,0.7705,"있는, 내년, 이라고, 미국, 것으로, 하는, 현재, 것은, 것이다, 있다는",[ 차은지 기자] 금융당국의 신용카드 수수료 적격비용 산정 결과 발표가 임박한 가운...
60,137,3.0,0.7360,"통해, 밝혔다, 재배포_금지, 관계자는, 있는, 기자, 억원을, 직접, 금지, 무단_전재",[ 김순신 기자] J트러스트그룹(JT캐피탈·JT친애저축은행·JT저축은행)은 지난 1...


,title,link,content
200,NaN,NaN,NaN
614,"""한 해 평균 370명이 과로사..탄력근로제는 과로사 양산제도""",http://m.finance.daum.net/m/news/news_content....,"(서울=연합뉴스) 김기훈 현혜란 기자 = ""탄력 근로제는 과로사를 양산하겠다는 제도..."
442,[코스피] 9.21p(0.44%) 내린 2071.23 마감,https://m.news.naver.com/read.nhn?oid=421&aid=...,13일 코스피 마감
429,최종구 위원장 '자동차 부품 견고하네',https://m.news.naver.com/read.nhn?oid=421&aid=...,(화성=뉴스1) 오장환 기자 = 최종구 금융위원회 위원장이 13일 오후 경기도 화성...
137,DGB그룹 편입 하이투자증권 '시너지 전략본부' 신설,https://m.news.naver.com/read.nhn?oid=001&aid=...,(서울=연합뉴스) 전명훈 기자 = DGB금융그룹 자회사로 편입된 하이투자증권이 DG...


In [27]:
recommendation_df = df.loc[recommendation_df.loc[duplicate_idx]['Document_No'].values].iloc[:,:2]
recommendation_df

,title,link
200,NaN,NaN
614,"""한 해 평균 370명이 과로사..탄력근로제는 과로사 양산제도""",http://m.finance.daum.net/m/news/news_content....
442,[코스피] 9.21p(0.44%) 내린 2071.23 마감,https://m.news.naver.com/read.nhn?oid=421&aid=...
429,최종구 위원장 '자동차 부품 견고하네',https://m.news.naver.com/read.nhn?oid=421&aid=...
137,DGB그룹 편입 하이투자증권 '시너지 전략본부' 신설,https://m.news.naver.com/read.nhn?oid=001&aid=...


In [28]:
recommendation_df['link'].values

array([nan,
       'http://m.finance.daum.net/m/news/news_content.daum?type=main&sub_type=&docid=MD20181114114052785&section=&limit=30&page=2',
       'https://m.news.naver.com/read.nhn?oid=421&aid=0003690347&sid1=101&sid2=259&mode=LS2D',
       'https://m.news.naver.com/read.nhn?oid=421&aid=0003690355&sid1=101&sid2=259&mode=LS2D',
       'https://m.news.naver.com/read.nhn?oid=001&aid=0010466883&sid1=101&sid2=259&mode=LS2D'],
      dtype=object)

# Telegram Service

In [31]:
import telegram
import pickle

my_token = pickle.load( open('pw.pickle', 'rb'))
my_token = my_token

bot = telegram.Bot(token = my_token) #봇을 생성합니다.

bot.sendMessage(chat_id='@SSU_TOP_5', text='안녕하세요. 좋은 아침입니다. {}토픽 별 기사 5개 추천해드리겠습니다.'.format(\
str(datetime.datetime.now().month)+' 월 ' + str(datetime.datetime.now().day)+' 일 '))

for i in range(len(recommendation_df)) : 
    recommendation = recommendation_df.iloc[i].values[1]
    bot.sendMessage(chat_id='@SSU_TOP_5', text=recommendation) #@bill_chat 으로 메세지를 보냅니다.

________________

#### 각각의 토픽을 대표하는 문서찾기
- 가끔 토픽 키워드(단어)는 단지 토픽들을 구성하는 것에 그치지 않는 경우가 있다.
- 토픽을 이해하는 것을 넘어서, 토픽을 형성하는데 가장 많은 기여를 한 문서를 찾아낼 수도 있다.

In [50]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.8516,"기자, 있는, 한다, 대한, 경제, 정부가, 말했다, 서울, 밝혔다, 위해",(서울=연합뉴스) 김동현 기자 = 성윤모 산업통상자원부 장관이 12일 박용만 대한상...
1,1.0,0.8629,"있는, 위해, 통해, 위한, 계획이다, 블록체인, 글로벌, 예정이다, 대한, 등을",아세안 10개국 선정 대상도시(26곳). ⓒ국토부 정부가 세종과 부산을 중심으로 추...
2,2.0,0.8473,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",(서울=연합뉴스) 박용주 기자 = 미국과 중국의 무역분쟁으로 한국이 직격탄을 맞고 ...
3,3.0,0.8535,"기자, 있는, 중국, 통해, 무단_전재, 재배포_금지, 올해, 국내, 최대, 온라인",CJ제일제당이 가정간편식(HMR) 수산캔 전문 브랜드 ‘계절어보’의 라인업을 확대한...
4,4.0,0.8320,"지난해, 분기, 지난, 이후, 기자, 것으로, 올해, 서울, 말했다, 경기","[한겨레] 강신욱 통계청장은 지난해 2분기가 경기 정점이었을 가능성이 있다며, 이르..."


#### 문서를 넘어선 토픽 분배
- 마지막으로 우리는 해당 정보에서 어떤 것들이 가장 많이 거론되었는지를 토픽의 크기(volume)과 분포(distribution)로 이해할 수 있게 된다.

In [51]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.iloc[:10]

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,0.0,"기자, 있는, 한다, 대한, 경제, 정부가, 말했다, 서울, 밝혔다, 위해",257.0,0.2420
1,4.0,"지난해, 분기, 지난, 이후, 기자, 것으로, 올해, 서울, 말했다, 경기",214.0,0.2015
2,2.0,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",208.0,0.1959
3,1.0,"있는, 위해, 통해, 위한, 계획이다, 블록체인, 글로벌, 예정이다, 대한, 등을",225.0,0.2119
4,2.0,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",158.0,0.1488
5,3.0,"기자, 있는, 중국, 통해, 무단_전재, 재배포_금지, 올해, 국내, 최대, 온라인",NaN,NaN
6,3.0,"기자, 있는, 중국, 통해, 무단_전재, 재배포_금지, 올해, 국내, 최대, 온라인",NaN,NaN
7,2.0,"내년, 억원, 기자, 것으로, 미국, 경우, 국내, 대한, 것이다, 이날",NaN,NaN
8,0.0,"기자, 있는, 한다, 대한, 경제, 정부가, 말했다, 서울, 밝혔다, 위해",NaN,NaN
9,4.0,"지난해, 분기, 지난, 이후, 기자, 것으로, 올해, 서울, 말했다, 경기",NaN,NaN
